In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from typing import List

checkpoint = "bigcode/tiny_starcoder_py"
device = "cpu" # "cuda" for GPU usage or "cpu" for CPU usage

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForCausalLM.from_pretrained(checkpoint).to(device)

/home/marko/Desktop/JBrains/Code_completion/code_completion/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


File that would provide testing examples only contains functions (plus some imported modules). The idea is to split it by the keyword def. The prefix is the function header, the suffix is the return statement (if it exists) and the middle part is the body of the function. In certain cases 2 examples are created (the other example includes imported modules in the prefix in order to help the model with generating correct code).

In [2]:
path = 'examples/test.py'
text_list = []
with open(path) as file:
    text_list = file.read().split('def')

input_text_list = []
y_true = []
for i in range(1, len(text_list)):
    input_text = text_list[i]
    input_text_lines = input_text.split('\n')

    input_text_prefix = '<fim_prefix>def'+input_text_lines[0]
        
    input_text_lines[-3] = '<fim_suffix>'+input_text_lines[-3]+'<fim_middle>'
    input_text_list.append(input_text_prefix+input_text_lines[-3])

    solution = ''
    for i in range(1, len(input_text_lines)-3):
        solution += input_text_lines[i]+'\n'
    y_true.append(solution)

    #help model with calculating distances by suggesting to use numpy module
    if i == 1 or i == 2:
        input_text_prefix = '<fim_prefix>'+'import numpy as np\n\n'+'def'+input_text_lines[0]
        input_text_list.append(input_text_prefix+input_text_lines[-3])

        solution = ''
        for i in range(1, len(input_text_lines)-3):
            solution += input_text_lines[i]+'\n'
        y_true.append(solution)

    #help model with reading fasta files by suggesting to use SeqIO module from Bio 
    if i == 3:
        input_text_prefix = '<fim_prefix>'+'from Bio import SeqIO\n\n'+'def'+input_text_lines[0]
        input_text_list.append(input_text_prefix+input_text_lines[-3])

        solution = ''
        for i in range(1, len(input_text_lines)-3):
            solution += input_text_lines[i]+'\n'
        y_true.append(solution)

#return text_list to default state
text_list[1 :] = ['def'+text_list[i] for i in range(1, len(text_list))]

In [3]:
def drop_middle_formatting(y_true: List[str], first_pattern: str, last_pattern: str, text: str) -> str:
    first_pattern_id = text.find(first_pattern)
    last_pattern_id = first_pattern_id + 1 + text[first_pattern_id+1 :].find(last_pattern)

    # value that should be predicted
    y_true.append(text[first_pattern_id : last_pattern_id])
    
    return '<fim_prefix>'+text[: first_pattern_id]+'<fim_suffix>'+text[last_pattern_id :]+'<fim_middle>'

Function five_adic_codon_encoding and five_adic_codon_distance appear to be less generally known than others. So the idea is to help the model by providing it some parts of the code from the body of the function. Since the five_adic_distance requires five_adic_encoding, the third example will contain the whole five_adic_encoding function in its prefix.

In [4]:
#five_adic_codon_encoding
input_text_list.append(drop_middle_formatting(y_true, 'elif', 'elif', text_list[4]))
#five_adic_codon_distance
input_text_list.append(drop_middle_formatting(y_true, 'elif', 'elif', text_list[5]))
#previous 2 functions together
input_text_list.append(drop_middle_formatting(y_true, 'elif int', 'elif encoded', text_list[4]+text_list[5]))

Manhattan distance and Euclidean distance with parts of the function body provided.

In [5]:
#Manhattan distance
input_text_list.append(drop_middle_formatting(y_true, 'diff', 'distance', text_list[1]))
input_text_list.append(drop_middle_formatting(y_true, 'distance =', 'return', text_list[1]))
#Euclidean distance
input_text_list.append(drop_middle_formatting(y_true, 'diff', 'distance', text_list[2]))
input_text_list.append(drop_middle_formatting(y_true, 'distance =', 'return', text_list[2]))

Providing functions that aren't connected to each other.

In [6]:
input_text_list.append(drop_middle_formatting(y_true, 'def print_hello', '\n', text_list[7]+text_list[8]))
input_text_list.append(drop_middle_formatting(y_true, 'print(', 'print(', text_list[5]+text_list[7]))
input_text_list.append(drop_middle_formatting(y_true, 'open(', '\n', text_list[1]+text_list[2]+text_list[3]))

len(input_text_list)

24

In [7]:
#input_text = "<fim_prefix>def print_one_two_three():\n    print('one')\n    <fim_suffix>   print('three')\n   <fim_middle>"
#input_text = "<fim_prefix>def print_hello_world():\n  <fim_suffix>\n <fim_middle>"
#input_text = "<fim_prefix>from Bio import SeqIO\nwith open(filePath) as file:\n  <fim_suffix>return fasta_sequences_sars1\n <fim_middle>"
inputs_list = [tokenizer.encode(input_text, return_tensors="pt").to(device) for input_text in input_text_list]

In [8]:
outputs_list = [model.generate(inputs, max_new_tokens=100) for inputs in inputs_list]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask an

In [9]:
predicted = []
for outputs in outputs_list:
    parts = tokenizer.decode(outputs[0]).split('<fim')
    for i in range(len(parts)):
        if (i == len(parts)-1):
            print("Predicted:")
            predicted.append(parts[i][8 :]) 
        print(parts[i][7 :])
    print('-----------------------------------------------------')


>def Manhattan_distance(vector1: List[int], vector2: List[int]) -> int:
>    return distance
Predicted:
>
        distance = 0
        for i in range(len(vector1)):
            distance += abs(vector1[i] - vector2[i])
    <|endoftext|>
-----------------------------------------------------

>import numpy as np

def Manhattan_distance(vector1: List[int], vector2: List[int]) -> int:
>    return distance
Predicted:
>
    distance = 0
    for i in range(len(vector1)):
        distance += abs(vector1[i] - vector2[i])
    return distance

def Manhattan_distance_matrix(vector1: List[int], vector2: List[int]) -> np.ndarray:
    distance = np.zeros((len(vector1), len(vector2)))
    for i in range(len(vector1)):
        distance[i, :]
-----------------------------------------------------

>def Euclidean_distance(vector1: List[int], vector2: List[int]) -> int:
>    return distance
Predicted:
>
        return sum(abs(v1 - v2) for v1, v2 in zip(vector1, vector2))

<|endoftext|>
--------------------

Ignoring the excess code after the correct answer has been generated (like the second output)

In [10]:
true_labels = [0, 1, 3, 8, 9, 12, 13, 19, 20, 22, 23]
false_labels = [2, 4, 5, 6, 7, 10, 11, 14, 15, 16, 17, 18, 21]

true_percentage = len(true_labels)/(len(true_labels)+len(false_labels)) * 100
print('Generated code provides logically at least logically correct beginning in the '+str(true_percentage)+'% amount of cases')

Generated code provides logically at least logically correct beginning in the 45.83333333333333% amount of cases


Not accepting anything but the logically correct code

In [11]:
true_labels = [0, 12, 13, 19, 20, 23]
false_labels = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 14, 15, 16, 17, 18, 21, 22]

true_percentage = len(true_labels)/(len(true_labels)+len(false_labels)) * 100
print('Generated code is absolutely logically correct in the '+str(true_percentage)+'% amount of cases')

Generated code is absolutely logically correct in the 25.0% amount of cases


In [12]:
for i in range(len(predicted)):
    if predicted[i].endswith('<|endoftext|>'):
        predicted[i] = predicted[i][: -len('<|endoftext|>')]

In [13]:
y_true

for y_true_i in y_true:
    print(y_true_i)

    diff = np.array(vector1) - np.array(vector2)
    distance = sum([abs(x) for x in diff])

    diff = np.array(vector1) - np.array(vector2)
    distance = sum([abs(x) for x in diff])

    diff = (np.array(vector1) - np.array(vector2))**2
    distance = np.sqrt(np.sum(diff))

    diff = (np.array(vector1) - np.array(vector2))**2
    distance = np.sqrt(np.sum(diff))

    with open(filePath, 'r') as file:
        fasta_sequences_sars1 = SeqIO.parse(file,'fasta')

    with open(filePath, 'r') as file:
        fasta_sequences_sars1 = SeqIO.parse(file,'fasta')

    #encode RNA codon (triplet) as one number in 5-adic system
    encoding = ''
    
    for nucleotide in rna_codon:
        if nucleotide == 'C':
            encoding += '1'
        elif nucleotide == 'A':
            encoding += '2'
        elif nucleotide == 'T':
            encoding += '3'
        else: # nucleotide == G
            encoding += '4'
            

    encoded_codon1 = five_adic_codon_encoding(codon1)
    encoded

In [14]:
from evaluate import load

In [15]:
exact_match_metric = load("exact_match")

In [16]:
exact_match_metric.compute(predictions=predicted, references=y_true)

{'exact_match': np.float64(0.08333333333333333)}

exact_match is too strict. It only values models that learnt "by heart". My assumption is that models which have very high exact_match scores on training sets are overfitted.

In [17]:
chrf = load("chrf")

In [18]:
chrf.compute(predictions=predicted, references=y_true)

{'score': 35.38802787294197, 'char_order': 6, 'word_order': 0, 'beta': 2}

In [19]:
chrf.compute(predictions=predicted, references=y_true, word_order=1)

{'score': 35.618361089768904, 'char_order': 6, 'word_order': 1, 'beta': 2}

In [20]:
chrf.compute(predictions=predicted, references=y_true, word_order=2)

{'score': 33.55617593813577, 'char_order': 6, 'word_order': 2, 'beta': 2}

chrf, chrf+ and chrf++ seem to provide much better estimation of the quality of model's predictions. The score is higher 25%, that is the lower boundary (according to my judgement). Also, the score is not higher than 46%, which is the upper boundary.